In [23]:

from torchvision.transforms.functional import to_pil_image

from torchcam.cams import ScoreCAM
from torchcam.utils import overlay_mask

import cv2
from tqdm import tqdm
import time

import numpy as np
from PIL import Image
from torchvision import models, transforms

import torch

from torchvision.transforms import Compose
import os
import torch.nn as nn

from imutils import paths

import matplotlib.pyplot as plt

from pfe.preprocessing.bonsup.resnetbs import resnet_bs

from pfe.preprocessing.segmentation.VAE import uVAE
from pfe.utils.utils import saveMask, loadDCM

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [25]:
resnet18masked = models.resnet18(pretrained=True)

resnet18masked.fc = nn.Sequential(
                    nn.Linear(512, 256),
                    nn.ReLU(),
                    nn.Linear(256, 4))
resnet18masked.to(device)
resnet18masked.load_state_dict(torch.load('pretrained_models/xresnet18masked.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [26]:
resnetbs = resnet_bs(num_filters=64, num_res_blocks=16, res_block_scaling=0.1)


In [27]:

VAE = uVAE(nhid= 16, nlatent=8)
VAE.load_state_dict(torch.load('pretrained_models/VAE.pt', map_location=torch.device("cpu")))
VAE.to(device)
VAE.eval()

resnetbs.load_weights('pretrained_models/resnet-bs.h5')

In [28]:
#path = "/content/gdrive/MyDrive/Our PFE/PFE/Data/OurData/Mendeley/Data/PNEUMONIA"
#img_paths = list(paths.list_images(path))
#len(img_paths)

In [29]:
def plot_hm(image):
  y_test_pred = resnet18masked(image.unsqueeze(0).to(device))
  _, y_pred_tags = torch.max(y_test_pred, dim = 1)
  activation_map = cam_extractor(y_test_pred.squeeze(0).argmax().item(), y_test_pred)
  result = overlay_mask(to_pil_image(image), to_pil_image(activation_map, mode='F'), alpha=0.5)
  plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()

In [43]:
image_list = ['/Users/benx13/PycharmProjects/pythonProject/pfe/images/chexnet.png']

In [44]:
i = 0
cam_extractor = ScoreCAM(resnet18masked)
for imgp in tqdm(image_list):
  print(str(i)+" "+imgp.split(os.path.sep)[-1])
  i+=1

  x = cv2.imread(imgp, cv2.IMREAD_GRAYSCALE)
  x = cv2.resize(x, (256,256))
  x = x.astype('float32') / 255

  x1 = np.expand_dims(x, axis=0)
  t=time.time()

  bs = resnetbs.predict(x1)
  bs = np.reshape(bs, (256,256))
  bs = bs*255

  img, roi, h, w, hLoc, wLoc, imH, imW = loadDCM(bs, preprocess=False)

  img = img.to(device)
  _,pred = VAE(img)
  pred = torch.sigmoid(pred.cpu()*roi)
  pred = saveMask(pred.squeeze(),h,w,hLoc,wLoc,imH,imW)
  pred = torch.from_numpy(pred)
  pred = torch.unsqueeze(pred, 0)
  pred = torch.unsqueeze(pred, 1)
  pred = pred.squeeze().numpy()

  pred = pred[65:510]
  pred = 1.0 * (pred > 0.5)

  img = img.cpu().squeeze()[65:510]
  img = cv2.resize(np.array(img), (256, 256))

  pred = cv2.resize(np.array(pred), (256, 256))

  bsmasked = img*pred*255
  bsmasked = cv2.resize(np.array(bsmasked), (256, 256))

  masked = x*pred*255
  masked = cv2.resize(np.array(masked), (256, 256))

  x = x*255

  t = Compose([transforms.ToPILImage(), transforms.Resize((224,224), interpolation=Image.NEAREST), transforms.ToTensor()])

  masked = masked.astype('uint8')
  bs = bs.astype('uint8')
  x = x.astype('uint8')
  bsmasked = bsmasked.astype('uint8')

  masked = t(np.stack((np.array(masked),)*3, axis=-1))
  bsmasked = t(np.stack((np.array(bsmasked),)*3, axis=-1))
  bs = t(np.stack((np.array(bs),)*3, axis=-1))
  x = t(np.stack((np.array(x),)*3, axis=-1))


  print('-----og-pred----')
  plot_hm(t(x))
  print('-----masked-pred----')
  plot_hm(t(masked))
  print('-----bs-pred----')
  plot_hm(t(bs))
  print('-----bs-masked-pred----')
  plot_hm(t(bsmasked))

  break

  0%|          | 0/1 [00:00<?, ?it/s]

0 chexnet.png


  0%|          | 0/1 [00:03<?, ?it/s]


OSError: Cannot understand given URI: array([[16.732504  ,  4.6807103 ,  0.76154566, ..., 14.58....